In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models, optimizers, callbacks
from tensorflow.keras.applications import ResNet50
import matplotlib.pyplot as plt
import numpy as np
import os


In [2]:
# Loading datasets; identical to notebook 01
DATA_DIR = '../data/asl_alphabet_train/asl_alphabet_train'
TEST_DATA_DIR = '../data/asl_alphabet_test/asl_alphabet_test'

IMAGE_SIZE = (96, 96)
BATCH_SIZE = 32
VALIDATION_SPLIT = 0.2
SEED = 123
NUM_CLASSES = 29  # A-Z, del, nothing, space

print("Loading training and validation datasets...")
train_ds = tf.keras.utils.image_dataset_from_directory(
    DATA_DIR,
    validation_split=VALIDATION_SPLIT,
    subset="training",
    seed=SEED,
    image_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    label_mode='categorical'
)

val_ds = tf.keras.utils.image_dataset_from_directory(
    DATA_DIR,
    validation_split=VALIDATION_SPLIT,
    subset="validation",
    seed=SEED,
    image_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    label_mode='categorical'
)

class_names = train_ds.class_names
print(f"Found {len(class_names)} classes: {class_names}")


Loading training and validation datasets...
Found 86912 files belonging to 29 classes.
Using 69530 files for training.
Found 86912 files belonging to 29 classes.
Using 17382 files for validation.
Found 29 classes: ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'del', 'nothing', 'space']


In [3]:
# Data augmentation and normalization
print("Defining augmentation and normalization layers...")

data_augmentation = tf.keras.Sequential(
    [
        layers.RandomFlip("horizontal"),
        layers.RandomRotation(0.1),
        layers.RandomZoom(0.1),
        layers.RandomTranslation(0.1, 0.1),
        layers.RandomContrast(0.1),
    ],
    name="data_augmentation"
)

normalization_layer = layers.Rescaling(1./255)

# Apply augmentation to training data, normalization to both
print("Applying processing and optimizing datasets...")

train_ds = train_ds.map(lambda x, y: (data_augmentation(x, training=True), y), num_parallel_calls=tf.data.AUTOTUNE)
train_ds = train_ds.map(lambda x, y: (normalization_layer(x), y), num_parallel_calls=tf.data.AUTOTUNE)

val_ds = val_ds.map(lambda x, y: (normalization_layer(x), y), num_parallel_calls=tf.data.AUTOTUNE)

AUTOTUNE = tf.data.AUTOTUNE
train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

print("Data processing pipeline complete and optimized.")

Defining augmentation and normalization layers...
Applying processing and optimizing datasets...
Data processing pipeline complete and optimized.


In [4]:
#Loading in ResNet50 and then freezing (not changing the weights) of the base model
base_model = ResNet50(
    weights='imagenet',     
    include_top=False,        
    input_shape=(96, 96, 3)  
)

base_model.trainable = False
print(f"  Number of layers: {len(base_model.layers)}")
print(f"  Total parameters: {base_model.count_params():,}")
print(f"  Trainable layers: {sum([1 for layer in base_model.layers if layer.trainable])}")


94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 2s 0us/step
  Number of layers: 175
  Total parameters: 23,587,712
  Trainable layers: 0


In [5]:
#Running images through the base model
inputs = keras.Input(shape=(96, 96, 3))
x = base_model(inputs, training=False)
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dropout(0.2)(x)
x = layers.Dense(128, activation='relu')(x)
x = layers.Dropout(0.2)(x)

outputs = layers.Dense(NUM_CLASSES, activation='softmax')(x)

model = models.Model(inputs, outputs, name='asl_resnet50')
model.summary()


Model: "asl_resnet50"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_2 (InputLayer)      │ (None, 96, 96, 3)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ resnet50 (Functional)           │ (None, 3, 3, 2048)     │    23,587,712 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d        │ (None, 2048)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 2048)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 128)            │       262,272 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 29)             │         3,741 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 23,853,725 (90.99 MB)

 Trainable params: 266,013 (1.01 MB)

 Non-trainable params: 23,587,712 (89.98 MB)

In [6]:
#Preparing the model for training
model.compile(
    optimizer=optimizers.Adam(learning_rate=0.001),  
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

## Phase 1: Training New Layers (Base Model Frozen)


In [7]:
callbacks_list = [
    #Early stopping to save time
    callbacks.EarlyStopping(
        monitor='val_loss',
        patience=5,
        restore_best_weights=True,
        verbose=1
    ),
    #Slowing down learning rate when stuck
    callbacks.ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.5,
        patience=3,
        min_lr=1e-7,
        verbose=1
    ),
    # Saving best model
    callbacks.ModelCheckpoint(
        'best_asl_resnet50_phase1.h5',
        monitor='val_accuracy',
        save_best_only=True,
        verbose=1
    )
]



In [8]:
#Training the new layers
history_phase1 = model.fit(
    train_ds,
    epochs=10,
    validation_data=val_ds,
    callbacks=callbacks_list,
    verbose=1
)

Epoch 1/10
2173/2173 ━━━━━━━━━━━━━━━━━━━━ 0s 86ms/step - accuracy: 0.0604 - loss: 3.3253
Epoch 1: val_accuracy improved from None to 0.15625, saving model to best_asl_resnet50_phase1.h5


2173/2173 ━━━━━━━━━━━━━━━━━━━━ 233s 106ms/step - accuracy: 0.0815 - loss: 3.2458 - val_accuracy: 0.1563 - val_loss: 2.9772 - learning_rate: 0.0010
Epoch 2/10
2173/2173 ━━━━━━━━━━━━━━━━━━━━ 0s 81ms/step - accuracy: 0.1156 - loss: 3.0876
Epoch 2: val_accuracy improved from 0.15625 to 0.19952, saving model to best_asl_resnet50_phase1.h5


2173/2173 ━━━━━━━━━━━━━━━━━━━━ 219s 101ms/step - accuracy: 0.1217 - loss: 3.0595 - val_accuracy: 0.1995 - val_loss: 2.7860 - learning_rate: 0.0010
Epoch 3/10
2173/2173 ━━━━━━━━━━━━━━━━━━━━ 0s 84ms/step - accuracy: 0.1395 - loss: 2.9896
Epoch 3: val_accuracy improved from 0.19952 to 0.21637, saving model to best_asl_resnet50_phase1.h5


2173/2173 ━━━━━━━━━━━━━━━━━━━━ 230s 106ms/step - accuracy: 0.1384 - loss: 2.9806 - val_accuracy: 0.2164 - val_loss: 2.6945 - learning_rate: 0.0010
Epoch 4/10
2173/2173 ━━━━━━━━━━━━━━━━━━━━ 0s 87ms/step - accuracy: 0.1467 - loss: 2.9443
Epoch 4: val_accuracy improved from 0.21637 to 0.23559, saving model to best_asl_resnet50_phase1.h5


2173/2173 ━━━━━━━━━━━━━━━━━━━━ 236s 109ms/step - accuracy: 0.1480 - loss: 2.9364 - val_accuracy: 0.2356 - val_loss: 2.6333 - learning_rate: 0.0010
Epoch 5/10
2173/2173 ━━━━━━━━━━━━━━━━━━━━ 0s 91ms/step - accuracy: 0.1524 - loss: 2.9113
Epoch 5: val_accuracy improved from 0.23559 to 0.25319, saving model to best_asl_resnet50_phase1.h5


2173/2173 ━━━━━━━━━━━━━━━━━━━━ 242s 111ms/step - accuracy: 0.1536 - loss: 2.9038 - val_accuracy: 0.2532 - val_loss: 2.5831 - learning_rate: 0.0010
Epoch 6/10
2173/2173 ━━━━━━━━━━━━━━━━━━━━ 0s 85ms/step - accuracy: 0.1589 - loss: 2.8822
Epoch 6: val_accuracy improved from 0.25319 to 0.25641, saving model to best_asl_resnet50_phase1.h5


2173/2173 ━━━━━━━━━━━━━━━━━━━━ 229s 105ms/step - accuracy: 0.1594 - loss: 2.8799 - val_accuracy: 0.2564 - val_loss: 2.5786 - learning_rate: 0.0010
Epoch 7/10
2173/2173 ━━━━━━━━━━━━━━━━━━━━ 0s 80ms/step - accuracy: 0.1630 - loss: 2.8602
Epoch 7: val_accuracy did not improve from 0.25641
2173/2173 ━━━━━━━━━━━━━━━━━━━━ 217s 100ms/step - accuracy: 0.1633 - loss: 2.8623 - val_accuracy: 0.2554 - val_loss: 2.5454 - learning_rate: 0.0010
Epoch 8/10
2173/2173 ━━━━━━━━━━━━━━━━━━━━ 0s 80ms/step - accuracy: 0.1688 - loss: 2.8444
Epoch 8: val_accuracy did not improve from 0.25641
2173/2173 ━━━━━━━━━━━━━━━━━━━━ 216s 100ms/step - accuracy: 0.1667 - loss: 2.8462 - val_accuracy: 0.2550 - val_loss: 2.5074 - learning_rate: 0.0010
Epoch 9/10
2173/2173 ━━━━━━━━━━━━━━━━━━━━ 0s 81ms/step - accuracy: 0.1670 - loss: 2.8363
Epoch 9: val_accuracy improved from 0.25641 to 0.25935, saving model to best_asl_resnet50_phase1.h5


2173/2173 ━━━━━━━━━━━━━━━━━━━━ 219s 101ms/step - accuracy: 0.1680 - loss: 2.8353 - val_accuracy: 0.2593 - val_loss: 2.4927 - learning_rate: 0.0010
Epoch 10/10
2173/2173 ━━━━━━━━━━━━━━━━━━━━ 0s 80ms/step - accuracy: 0.1744 - loss: 2.8203
Epoch 10: val_accuracy improved from 0.25935 to 0.26424, saving model to best_asl_resnet50_phase1.h5


2173/2173 ━━━━━━━━━━━━━━━━━━━━ 218s 100ms/step - accuracy: 0.1735 - loss: 2.8222 - val_accuracy: 0.2642 - val_loss: 2.4864 - learning_rate: 0.0010
Restoring model weights from the end of the best epoch: 10.


## Phase 2: Fine-Tuning Entire Model


In [9]:
#Unfreezing the base model
base_model.trainable = True
for layer in base_model.layers[:143]:
    layer.trainable = False

model.compile(
    optimizer=optimizers.Adam(learning_rate=0.0001),  # 10x lower LR
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

print(f"Total trainable parameters now: {sum([tf.size(w).numpy() for w in model.trainable_weights]):,}")


Total trainable parameters now: 15,242,013


In [20]:
print("="*60)
print("STARTING PHASE 2 FINE-TUNING")
print("="*60)
print(f"Trainable parameters: {sum([tf.size(w).numpy() for w in model.trainable_weights]):,}")
print("This will take approximately 30-60 minutes...")
print()

# Setup callbacks for Phase 2
callbacks_list_phase2 = [
    callbacks.EarlyStopping(
        monitor='val_loss',
        patience=5,
        restore_best_weights=True,
        verbose=1
    ),
    callbacks.ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.5,
        patience=3,
        min_lr=1e-7,
        verbose=1
    ),
    callbacks.ModelCheckpoint(
        'best_asl_resnet50_phase2.h5',
        monitor='val_accuracy',
        save_best_only=True,
        verbose=1
    )
]

# FIXED: Train Phase 2 properly (NO initial_epoch!)
history_phase2 = model.fit(
    train_ds,
    epochs=10,  
    validation_data=val_ds,
    callbacks=callbacks_list_phase2,
    verbose=1
)

print("\n" + "="*60)
print("✓ PHASE 2 FINE-TUNING COMPLETE!")
print("="*60)

STARTING PHASE 2 FINE-TUNING
Trainable parameters: 15,242,013
This will take approximately 30-60 minutes...

Epoch 1/10
2173/2173 ━━━━━━━━━━━━━━━━━━━━ 0s 174ms/step - accuracy: 0.1147 - loss: 4.1229
Epoch 1: val_accuracy improved from None to 0.28760, saving model to best_asl_resnet50_phase2.h5


2173/2173 ━━━━━━━━━━━━━━━━━━━━ 426s 194ms/step - accuracy: 0.1616 - loss: 3.0791 - val_accuracy: 0.2876 - val_loss: 2.4233 - learning_rate: 1.0000e-04
Epoch 2/10
2173/2173 ━━━━━━━━━━━━━━━━━━━━ 0s 169ms/step - accuracy: 0.2551 - loss: 2.5059
Epoch 2: val_accuracy improved from 0.28760 to 0.37982, saving model to best_asl_resnet50_phase2.h5


2173/2173 ━━━━━━━━━━━━━━━━━━━━ 410s 189ms/step - accuracy: 0.2697 - loss: 2.4399 - val_accuracy: 0.3798 - val_loss: 2.0520 - learning_rate: 1.0000e-04
Epoch 3/10
2173/2173 ━━━━━━━━━━━━━━━━━━━━ 0s 176ms/step - accuracy: 0.3121 - loss: 2.2684
Epoch 3: val_accuracy did not improve from 0.37982
2173/2173 ━━━━━━━━━━━━━━━━━━━━ 428s 197ms/step - accuracy: 0.3204 - loss: 2.2283 - val_accuracy: 0.3570 - val_loss: 2.0954 - learning_rate: 1.0000e-04
Epoch 4/10
2173/2173 ━━━━━━━━━━━━━━━━━━━━ 0s 174ms/step - accuracy: 0.3502 - loss: 2.1100
Epoch 4: val_accuracy did not improve from 0.37982
2173/2173 ━━━━━━━━━━━━━━━━━━━━ 421s 194ms/step - accuracy: 0.3603 - loss: 2.0715 - val_accuracy: 0.3194 - val_loss: 2.7869 - learning_rate: 1.0000e-04
Epoch 5/10
2173/2173 ━━━━━━━━━━━━━━━━━━━━ 0s 170ms/step - accuracy: 0.3937 - loss: 1.9533
Epoch 5: val_accuracy improved from 0.37982 to 0.39455, saving model to best_asl_resnet50_phase2.h5


2173/2173 ━━━━━━━━━━━━━━━━━━━━ 415s 191ms/step - accuracy: 0.4029 - loss: 1.9209 - val_accuracy: 0.3945 - val_loss: 1.9186 - learning_rate: 1.0000e-04
Epoch 6/10
2173/2173 ━━━━━━━━━━━━━━━━━━━━ 0s 172ms/step - accuracy: 0.4330 - loss: 1.8087
Epoch 6: val_accuracy improved from 0.39455 to 0.42130, saving model to best_asl_resnet50_phase2.h5


2173/2173 ━━━━━━━━━━━━━━━━━━━━ 418s 192ms/step - accuracy: 0.4427 - loss: 1.7732 - val_accuracy: 0.4213 - val_loss: 2.4739 - learning_rate: 1.0000e-04
Epoch 7/10
2173/2173 ━━━━━━━━━━━━━━━━━━━━ 0s 173ms/step - accuracy: 0.4714 - loss: 1.6725
Epoch 7: val_accuracy improved from 0.42130 to 0.48061, saving model to best_asl_resnet50_phase2.h5


2173/2173 ━━━━━━━━━━━━━━━━━━━━ 421s 194ms/step - accuracy: 0.4790 - loss: 1.6436 - val_accuracy: 0.4806 - val_loss: 1.7146 - learning_rate: 1.0000e-04
Epoch 8/10
2173/2173 ━━━━━━━━━━━━━━━━━━━━ 0s 175ms/step - accuracy: 0.5088 - loss: 1.5451
Epoch 8: val_accuracy did not improve from 0.48061
2173/2173 ━━━━━━━━━━━━━━━━━━━━ 424s 195ms/step - accuracy: 0.5164 - loss: 1.5221 - val_accuracy: 0.3921 - val_loss: 2.1988 - learning_rate: 1.0000e-04
Epoch 9/10
2173/2173 ━━━━━━━━━━━━━━━━━━━━ 0s 173ms/step - accuracy: 0.5402 - loss: 1.4395
Epoch 9: val_accuracy did not improve from 0.48061
2173/2173 ━━━━━━━━━━━━━━━━━━━━ 419s 193ms/step - accuracy: 0.5469 - loss: 1.4175 - val_accuracy: 0.4228 - val_loss: 2.3608 - learning_rate: 1.0000e-04
Epoch 10/10
2173/2173 ━━━━━━━━━━━━━━━━━━━━ 0s 170ms/step - accuracy: 0.5706 - loss: 1.3409
Epoch 10: ReduceLROnPlateau reducing learning rate to 4.999999873689376e-05.

Epoch 10: val_accuracy did not improve from 0.48061
2173/2173 ━━━━━━━━━━━━━━━━━━━━ 416s 191ms/st

## Training Results


In [21]:
# Display comprehensive results
print("\n" + "="*70)
print(" "*20 + "TRAINING RESULTS SUMMARY")
print("="*70)

print("\n📊 PHASE 1: Frozen Base Model (COMPLETED)")
print("-" * 70)
print(f"  Final Training Accuracy:    {history_phase1.history['accuracy'][-1]:.4f} ({history_phase1.history['accuracy'][-1]*100:.2f}%)")
print(f"  Final Validation Accuracy:  {history_phase1.history['val_accuracy'][-1]:.4f} ({history_phase1.history['val_accuracy'][-1]*100:.2f}%)")
print(f"  Final Training Loss:        {history_phase1.history['loss'][-1]:.4f}")
print(f"  Final Validation Loss:      {history_phase1.history['val_loss'][-1]:.4f}")
print(f"  Epochs Trained:             {len(history_phase1.epoch)}")

print("\n📊 PHASE 2: Fine-Tuned Model (COMPLETED)")
print("-" * 70)
print(f"  Final Training Accuracy:    {history_phase2.history['accuracy'][-1]:.4f} ({history_phase2.history['accuracy'][-1]*100:.2f}%)")
print(f"  Final Validation Accuracy:  {history_phase2.history['val_accuracy'][-1]:.4f} ({history_phase2.history['val_accuracy'][-1]*100:.2f}%)")
print(f"  Final Training Loss:        {history_phase2.history['loss'][-1]:.4f}")
print(f"  Final Validation Loss:      {history_phase2.history['val_loss'][-1]:.4f}")
print(f"  Epochs Trained:             {len(history_phase2.epoch)}")

# Calculate improvement
phase1_val_acc = history_phase1.history['val_accuracy'][-1]
phase2_val_acc = history_phase2.history['val_accuracy'][-1]
improvement = (phase2_val_acc - phase1_val_acc) * 100

print("\n🎯 IMPROVEMENT")
print("-" * 70)
print(f"  Validation Accuracy Gain:   +{improvement:.2f}%")
print(f"  (From {phase1_val_acc*100:.2f}% → {phase2_val_acc*100:.2f}%)")

print("\n" + "="*70)
print("✓ TRAINING PIPELINE COMPLETE!")
print("="*70)



                    TRAINING RESULTS SUMMARY

📊 PHASE 1: Frozen Base Model (COMPLETED)
----------------------------------------------------------------------
  Final Training Accuracy:    0.1735 (17.35%)
  Final Validation Accuracy:  0.2642 (26.42%)
  Final Training Loss:        2.8222
  Final Validation Loss:      2.4864
  Epochs Trained:             10

📊 PHASE 2: Fine-Tuned Model (COMPLETED)
----------------------------------------------------------------------
  Final Training Accuracy:    0.5747 (57.47%)
  Final Validation Accuracy:  0.4724 (47.24%)
  Final Training Loss:        1.3242
  Final Validation Loss:      1.8604
  Epochs Trained:             10

🎯 IMPROVEMENT
----------------------------------------------------------------------
  Validation Accuracy Gain:   +20.82%
  (From 26.42% → 47.24%)

✓ TRAINING PIPELINE COMPLETE!


In [22]:
# Save the final trained model
model.save('asl_resnet50_final.keras')
print("✓ Final model saved as 'asl_resnet50_final.keras'")

# Verify all models
print("\n📁 Final Model Inventory:")
print("-" * 60)
for filename in ['best_asl_resnet50_phase1.h5', 
                 'best_asl_resnet50_phase2.h5',
                 'asl_resnet50_final.keras']:
    if os.path.exists(filename):
        size_mb = os.path.getsize(filename) / (1024*1024)
        print(f"  ✓ {filename:40s} ({size_mb:>6.1f} MB)")
    else:
        print(f"  ✗ {filename:40s} (NOT FOUND)")

print("\n✓ All done! Ready for evaluation in Notebook 07.")


✓ Final model saved as 'asl_resnet50_final.keras'

📁 Final Model Inventory:
------------------------------------------------------------
  ✓ best_asl_resnet50_phase1.h5              (  93.4 MB)
  ✓ best_asl_resnet50_phase2.h5              ( 207.7 MB)
  ✓ asl_resnet50_final.keras                 ( 207.9 MB)

✓ All done! Ready for evaluation in Notebook 07.
